<a href="https://colab.research.google.com/github/MokshithReddy17/simple-blockchain/blob/main/blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("hello")

hello


In [6]:
import hashlib
import time
import json

class Block:
    def __init__(self, index, transactions, previous_hash, nonce=0):
        self.index = index
        self.timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.hash = self.compute_hash()

    def compute_hash(self):
        block_data = json.dumps({
            "index": self.index,
            "timestamp": self.timestamp,
            "transactions": self.transactions,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce
        }, sort_keys=True)
        return hashlib.sha256(block_data.encode()).hexdigest()
    #Implementation of a basic proof-of-work mechanism
    def mine_block(self, difficulty):
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.compute_hash()
        print(f"Block {self.index} mined: {self.hash}")


In [7]:
class Blockchain:
    def __init__(self, difficulty=4):
        self.chain = []
        self.pending_transactions = []
        self.difficulty = difficulty
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(0, [], "0")
        self.chain.append(genesis_block)

    def add_transaction(self, sender, receiver, amount):
        self.pending_transactions.append({"sender": sender, "receiver": receiver, "amount": amount})

    def add_block(self):
        if not self.pending_transactions:
            print("No transactions to add!")
            return

        previous_block = self.chain[-1]
        new_block = Block(len(self.chain), self.pending_transactions.copy(), previous_block.hash)
        new_block.mine_block(self.difficulty)
        self.chain.append(new_block)
        self.pending_transactions.clear()

    def is_valid(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]
            if current.hash != current.compute_hash():
                return False
            if current.previous_hash != previous.hash:
                return False
        return True

    def tamper_block(self, index, new_transactions):
        if 0 < index < len(self.chain):
            self.chain[index].transactions = new_transactions
            self.chain[index].hash = self.chain[index].compute_hash()

    def print_chain(self):
        for block in self.chain:
            print(json.dumps({
                "index": block.index,
                "timestamp": block.timestamp,
                "transactions": block.transactions,
                "previous_hash": block.previous_hash,
                "hash": block.hash,
                "nonce": block.nonce
            }, indent=4))

In [8]:
blockchain = Blockchain()
blockchain.add_transaction("Mokshith", "Vishal", 1000)
blockchain.add_block()
blockchain.add_transaction("Vishal", "Sharath", 300)
blockchain.add_block()
blockchain.print_chain()
print("Is blockchain valid?", blockchain.is_valid())

# Checking
blockchain.tamper_block(1, [{"sender": "Ruthwik", "receiver": "Mokshith", "amount": 1500}])
print("Is blockchain valid after tampering?", blockchain.is_valid())

Block 1 mined: 00009979f62bc8435440bdba124fd43dfe145fd5f58a7fb0cec157011d1ad160
Block 2 mined: 0000717089bcdc8fe55d6ec9b4fab507a0be90505dc40609a0aa67c9ee1b58ff
{
    "index": 0,
    "timestamp": "2025-02-10 15:59:35",
    "transactions": [],
    "previous_hash": "0",
    "hash": "60c311be1bdcb3ddd1af3d249c527bf4c6ceba64df3273faac9d1a2df3c60aca",
    "nonce": 0
}
{
    "index": 1,
    "timestamp": "2025-02-10 15:59:35",
    "transactions": [
        {
            "sender": "Mokshith",
            "receiver": "Vishal",
            "amount": 1000
        }
    ],
    "previous_hash": "60c311be1bdcb3ddd1af3d249c527bf4c6ceba64df3273faac9d1a2df3c60aca",
    "hash": "00009979f62bc8435440bdba124fd43dfe145fd5f58a7fb0cec157011d1ad160",
    "nonce": 5413
}
{
    "index": 2,
    "timestamp": "2025-02-10 15:59:35",
    "transactions": [
        {
            "sender": "Vishal",
            "receiver": "Sharath",
            "amount": 300
        }
    ],
    "previous_hash": "00009979f62bc8435440bd